In [ ]:
import os
import glob
import re
import numpy as np
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import segmentation_models_pytorch as smp
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
import albumentations as A
from albumentations.pytorch import ToTensorV2
import json
from datetime import datetime
import random
import pandas as pd
from collections import defaultdict
import shutil
from pathlib import Path

# 필요한 함수들 재정의 (원본 코드에서 가져옴)
def get_patient_id_from_path(path):
    """
    경로에서 환자 ID 추출
    """
    parts = path.split('/')
    for part in parts:
        if part.isdigit() and len(part) > 5:  # 환자 ID는 보통 긴 숫자
            return part
    return None

def get_frame_num_from_path(path):
    """
    경로에서 frame 번호 추출
    """
    match = re.search(r'frame_(\d+)\.png', path)
    if match:
        return int(match.group(1))
    return None

def calculate_dice_score(pred, target, smooth=1e-6):
    """
    Calculate Dice coefficient for segmentation results (foreground class only)
    
    Args:
        pred (torch.Tensor or numpy.ndarray): Predicted mask (B, H, W) or already argmaxed (B, H, W)
        target (torch.Tensor or numpy.ndarray): Ground truth mask (B, H, W)
        smooth (float): Small value to prevent division by zero
    
    Returns:
        float: Dice coefficient for foreground class (1) only
    """
    # Convert NumPy arrays to PyTorch tensors if needed
    if isinstance(pred, np.ndarray):
        pred = torch.from_numpy(pred)
    if isinstance(target, np.ndarray):
        target = torch.from_numpy(target)
    
    # Convert predictions from probability distribution to class indices if needed
    if len(pred.shape) == 4:  # (B, C, H, W) format
        pred = torch.argmax(pred, dim=1)  # Convert to (B, H, W) format
    
    # 크기 확인 및 조정
    if pred.shape != target.shape:
        print(f"경고: 예측 텐서와 타겟 텐서의 크기가 다릅니다. 예측: {pred.shape}, 타겟: {target.shape}")
        
        # 동일한 크기로 조정
        if len(pred.shape) == 3:  # (B, H, W) 형태
            # 배치 크기 유지하고 높이와 너비만 조정
            h, w = min(pred.shape[1], target.shape[1]), min(pred.shape[2], target.shape[2])
            pred = pred[:, :h, :w]
            target = target[:, :h, :w]
        elif len(pred.shape) == 2:  # (H, W) 형태
            # 높이와 너비 조정
            h, w = min(pred.shape[0], target.shape[0]), min(pred.shape[1], target.shape[1])
            pred = pred[:h, :w]
            target = target[:h, :w]
    
    # Calculate Dice coefficient for foreground (class 1)
    pred_fg = (pred == 1).float()
    target_fg = (target == 1).float()
    
    intersection = (pred_fg * target_fg).sum()
    union = pred_fg.sum() + target_fg.sum()
    
    dice = (2.0 * intersection + smooth) / (union + smooth)
    
    return dice.item()

class MRIDataset(Dataset):
    def __init__(self, triplets, transform=None, target_size=(224, 224)):
        """
        MRI Dataset Class
        
        Args:
            triplets (list): List of tuples in the form (image_path, mask_path)
            transform (albumentations.Compose): albumentations transformations
            target_size (tuple): Target image size (width, height)
        """
        self.triplets = triplets
        self.transform = transform
        self.target_size = target_size
        
    def __len__(self):
        return len(self.triplets)
    
    def __getitem__(self, idx):
        image_path, mask_path = self.triplets[idx]
        
        # 이미지 로드
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if image is None:
            image = np.zeros(self.target_size, dtype=np.uint8)
        elif image.shape != self.target_size:
            image = cv2.resize(image, self.target_size)
        
        # 마스크 로드
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE) if mask_path and os.path.exists(mask_path) else None
        if mask is None:
            mask = np.zeros(self.target_size, dtype=np.uint8)
        else:
            # 크기 조정
            if mask.shape != self.target_size:
                mask = cv2.resize(mask, self.target_size, interpolation=cv2.INTER_NEAREST)
        
        # 마스크가 0/255 값을 가질 경우 0, 1로 변환
        if mask.max() > 1:
            mask = (mask > 0).astype(np.uint8)
        
        # 전처리 및 데이터 증강
        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image_transformed = augmented['image']
            mask_transformed = augmented['mask']
            
            # float 타입 확인
            if isinstance(image_transformed, torch.Tensor):
                image_transformed = image_transformed.float()
        else:
            # 변환 없는 경우 기본 텐서 변환
            image_transformed = torch.from_numpy(image).float().unsqueeze(0) / 255.0
            mask_transformed = torch.from_numpy(mask).long()
        
        # 마스크가 텐서가 아니면 변환
        if not isinstance(mask_transformed, torch.Tensor):
            mask_transformed = torch.from_numpy(mask_transformed).long()
        
        return image_transformed, mask_transformed, image_path, mask_path

def create_image_mask_pairs(patient_frames, base_dir, modality="t1"):
    """
    환자별 프레임 정보에서 이미지-마스크 쌍 생성
    """
    # 학습/검증용 환자와 테스트용 환자 구분
    train_patients = [pid for pid, frame_info in patient_frames.items() if not frame_info["is_test"]]
    test_patients = [pid for pid, frame_info in patient_frames.items() if frame_info["is_test"]]
    
    # 경로 설정
    mask_dir = os.path.join(base_dir, "Mask")
    
    if modality == "t1":
        image_dir = os.path.join(base_dir, "Image_T1")
    elif modality == "t2":
        image_dir = os.path.join(base_dir, "Image_T2")
    elif modality == "junction":
        image_dir = os.path.join(base_dir, "Image_junction")
    elif modality == "extension":
        image_dir = os.path.join(base_dir, "Image_extension")
    else:
        raise ValueError(f"Invalid modality: {modality}")
    
    # 테스트 쌍 생성
    test_pairs = []
    for patient_id in test_patients:
        frame_info = patient_frames[patient_id]
        
        # 모든 유효한 프레임 (병변 + 비병변)
        all_frames = frame_info["lesion_frames"] + frame_info["non_lesion_frames"]
        
        for frame_num in all_frames:
            # 마스크 경로
            mask_path = os.path.join(mask_dir, patient_id, f"frame_{frame_num}.png")
            
            # 이미지 경로
            image_path = os.path.join(image_dir, patient_id, f"frame_{frame_num}.png")
            
            # 이미지와 마스크가 모두 있는 경우만 추가
            if os.path.exists(image_path) and os.path.exists(mask_path):
                test_pairs.append((image_path, mask_path))
    
    return test_pairs

def collect_patient_frames(base_dir, test_patient_ids, min_frame=20, max_frame=138):
    """
    모든 환자의 프레임 정보 수집
    """
    # 환자별 프레임 정보
    patient_frames = {}
    
    # 경로 설정
    mask_dir = os.path.join(base_dir, "Mask")
    t1_dir = os.path.join(base_dir, "Image_T1")
    t2_dir = os.path.join(base_dir, "Image_T2")
    junction_dir = os.path.join(base_dir, "Image_junction")
    extension_dir = os.path.join(base_dir, "Image_extension")
    
    # 모든 환자 ID 수집
    all_patient_ids = []
    
    # Mask 디렉토리에서 환자 ID 수집
    for dir_path in glob.glob(f"{mask_dir}/*"):
        if os.path.isdir(dir_path):
            patient_id = os.path.basename(dir_path)
            if patient_id.isdigit() and len(patient_id) > 5:  # 환자 ID 확인
                all_patient_ids.append(patient_id)
    
    all_patient_ids = sorted(list(set(all_patient_ids)))
    
    # 각 환자에 대해 모든 프레임 정보 수집
    for patient_id in all_patient_ids:
        # 환자가 테스트 세트에 있는지 확인
        is_test = patient_id in test_patient_ids
        
        # 환자 경로 설정
        patient_mask_dir = os.path.join(mask_dir, patient_id)
        patient_t1_dir = os.path.join(t1_dir, patient_id)
        patient_t2_dir = os.path.join(t2_dir, patient_id)
        patient_junction_dir = os.path.join(junction_dir, patient_id)
        patient_extension_dir = os.path.join(extension_dir, patient_id)
        
        # 각 모달리티의 프레임 수집
        t1_frames = set()
        if os.path.exists(patient_t1_dir):
            for file_path in glob.glob(f"{patient_t1_dir}/frame_*.png"):
                frame_num = get_frame_num_from_path(file_path)
                if frame_num and is_valid_frame(frame_num, min_frame, max_frame):
                    t1_frames.add(frame_num)
        
        t2_frames = set()
        if os.path.exists(patient_t2_dir):
            for file_path in glob.glob(f"{patient_t2_dir}/frame_*.png"):
                frame_num = get_frame_num_from_path(file_path)
                if frame_num and is_valid_frame(frame_num, min_frame, max_frame):
                    t2_frames.add(frame_num)
        
        junction_frames = set()
        if os.path.exists(patient_junction_dir):
            for file_path in glob.glob(f"{patient_junction_dir}/frame_*.png"):
                frame_num = get_frame_num_from_path(file_path)
                if frame_num and is_valid_frame(frame_num, min_frame, max_frame):
                    junction_frames.add(frame_num)
        
        extension_frames = set()
        if os.path.exists(patient_extension_dir):
            for file_path in glob.glob(f"{patient_extension_dir}/frame_*.png"):
                frame_num = get_frame_num_from_path(file_path)
                if frame_num and is_valid_frame(frame_num, min_frame, max_frame):
                    extension_frames.add(frame_num)
        
        # 마스크 프레임
        mask_frames = set()
        if os.path.exists(patient_mask_dir):
            for file_path in glob.glob(f"{patient_mask_dir}/frame_*.png"):
                frame_num = get_frame_num_from_path(file_path)
                if frame_num and is_valid_frame(frame_num, min_frame, max_frame):
                    mask_frames.add(frame_num)
        
        # 모든 모달리티에 프레임이 존재하는 경우만 유효한 프레임으로 간주
        valid_frames = t1_frames.intersection(t2_frames).intersection(junction_frames).intersection(extension_frames).intersection(mask_frames)
        
        # 병변 프레임 확인
        lesion_frames = set()
        for frame_num in valid_frames:
            mask_path = os.path.join(patient_mask_dir, f"frame_{frame_num}.png")
            if has_lesion(mask_path):
                lesion_frames.add(frame_num)
        
        # 유효한 프레임 중에서 병변 프레임과 비병변 프레임 구분
        valid_lesion_frames = lesion_frames
        valid_non_lesion_frames = valid_frames - valid_lesion_frames
        
        # 환자별 프레임 정보 저장
        patient_frames[patient_id] = {
            "total_frames": len(valid_frames),
            "lesion_frames": sorted(list(valid_lesion_frames)),
            "non_lesion_frames": sorted(list(valid_non_lesion_frames)),
            "t1_available": sorted(list(t1_frames.intersection(valid_frames))),
            "t2_available": sorted(list(t2_frames.intersection(valid_frames))),
            "junction_available": sorted(list(junction_frames.intersection(valid_frames))),
            "extension_available": sorted(list(extension_frames.intersection(valid_frames))),
            "mask_available": sorted(list(mask_frames.intersection(valid_frames))),
            "is_test": is_test
        }
    
    return patient_frames

def is_valid_frame(frame_num, min_frame=20, max_frame=138):
    """
    유효한 프레임 번호인지 확인
    """
    return min_frame <= frame_num <= max_frame

def has_lesion(image_path):
    """
    이미지에 병변이 있는지 확인
    """
    if not os.path.exists(image_path):
        return False
    
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        return False
    
    # 흰색 픽셀(픽셀 값 > 0)이 있는지 확인
    return np.any(img > 0)

# 결과 분석 및 시각화 함수들
def load_model(model_path, device):
    """
    저장된 모델 불러오기
    
    Args:
        model_path: 모델 가중치 파일 경로
        device: 디바이스 (CPU/GPU)
        
    Returns:
        loaded_model: 불러온 모델
    """
    model = smp.UnetPlusPlus(
        encoder_name="resnet50",
        encoder_weights="imagenet",
        in_channels=1,
        classes=2,
    )
    model.to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    
    return model

def predict_batch(model, batch_images, device):
    """
    배치 이미지에 대한 예측 수행
    
    Args:
        model: 모델
        batch_images: 배치 이미지 (B, C, H, W)
        device: 디바이스 (CPU/GPU)
        
    Returns:
        predictions: 예측 마스크 (B, H, W)
    """
    with torch.no_grad():
        batch_images = batch_images.to(device)
        outputs = model(batch_images)
        predictions = torch.argmax(outputs, dim=1).cpu().numpy()
    
    return predictions

def evaluate_model_on_test_data(model, test_loader, device):
    """
    테스트 데이터셋에 대한 모델 평가
    
    Args:
        model: 모델
        test_loader: 테스트 데이터 로더
        device: 디바이스 (CPU/GPU)
        
    Returns:
        results: 평가 결과 (각 이미지별 결과와 전체 평균)
    """
    model.eval()
    
    all_results = []
    dice_scores = []
    
    with torch.no_grad():
        for images, masks, image_paths, mask_paths in tqdm(test_loader, desc="Evaluating model"):
            # 이미지를 float 타입으로 변환 확인
            images = images.float().to(device)
            masks = masks.cpu().numpy()
            
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1).cpu().numpy()
            
            # 이미지별 Dice 점수 계산
            for i in range(len(images)):
                img_dice = calculate_dice_score(preds[i:i+1], masks[i:i+1])
                dice_scores.append(img_dice)
                
                patient_id = get_patient_id_from_path(image_paths[i])
                frame_num = get_frame_num_from_path(image_paths[i])
                
                all_results.append({
                    'patient_id': patient_id,
                    'frame_num': frame_num,
                    'image_path': image_paths[i],
                    'mask_path': mask_paths[i],
                    'dice_score': img_dice,
                    'pred': preds[i].copy(),
                })
    
    avg_dice = np.mean(dice_scores)
    
    results = {
        'individual_results': all_results,
        'avg_dice_score': avg_dice,
    }
    
    return results

def visualize_results(image, mask, pred, dice_score, title=None, save_path=None):
    """
    결과 시각화
    
    Args:
        image: 원본 이미지
        mask: GT 마스크
        pred: 예측 마스크
        dice_score: Dice 점수
        title: 제목
        save_path: 저장 경로
    """
    # 크기 확인 및 조정
    if image.shape != mask.shape or image.shape != pred.shape:
        print(f"경고: 이미지와 마스크/예측의 크기가 다릅니다.")
        print(f"이미지: {image.shape}, 마스크: {mask.shape}, 예측: {pred.shape}")
        
        # 최소 크기로 조정
        min_h = min(image.shape[0], mask.shape[0], pred.shape[0])
        min_w = min(image.shape[1], mask.shape[1], pred.shape[1])
        
        image = image[:min_h, :min_w]
        mask = mask[:min_h, :min_w]
        pred = pred[:min_h, :min_w]
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # 원본 이미지
    axes[0].imshow(image, cmap='gray')
    axes[0].set_title('Original Image')
    axes[0].axis('off')
    
    # GT 마스크
    axes[1].imshow(mask, cmap='nipy_spectral')
    axes[1].set_title('Ground Truth')
    axes[1].axis('off')
    
    # 예측 마스크
    axes[2].imshow(pred, cmap='nipy_spectral')
    axes[2].set_title(f'Prediction (Dice={dice_score:.4f})')
    axes[2].axis('off')
    
    if title:
        plt.suptitle(title)
    
    plt.tight_layout()
    
    if save_path:
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close()
    else:
        plt.show()

def load_and_prepare_image(image_path, mask_path, target_size=(224, 224)):
    """
    이미지와 마스크 로드 및 전처리
    
    Args:
        image_path: 이미지 경로
        mask_path: 마스크 경로
        target_size: 대상 크기 (width, height)
        
    Returns:
        orig_image: 원본 이미지 (H, W)
        mask: 마스크 (H, W)
        tensor_image: 텐서 변환된 이미지 [1, 1, H, W]
    """
    # 이미지 로드
    orig_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if orig_image is None:
        orig_image = np.zeros(target_size, dtype=np.uint8)
    
    # 마스크 로드
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE) if mask_path and os.path.exists(mask_path) else None
    if mask is None:
        mask = np.zeros(target_size, dtype=np.uint8)
    else:
        # 크기 조정
        if mask.shape != target_size:
            mask = cv2.resize(mask, target_size, interpolation=cv2.INTER_NEAREST)
    
    # 마스크가 0/255 값을 가질 경우 0, 1로 변환
    if mask.max() > 1:
        mask = (mask > 0).astype(np.uint8)
    
    # 이미지 전처리
    processed_image = orig_image.copy()
    if processed_image.shape != target_size:
        processed_image = cv2.resize(processed_image, target_size)
    
    # 텐서 변환
    tensor_image = torch.from_numpy(processed_image).float().unsqueeze(0).unsqueeze(0) / 255.0
    
    return orig_image, mask, tensor_image

def create_confusion_matrix(pred, gt):
    """
    혼동 행렬 생성
    
    Args:
        pred: 예측 마스크
        gt: GT 마스크
        
    Returns:
        cm: 혼동 행렬 (2x2)
    """
    if isinstance(pred, torch.Tensor):
        pred = pred.cpu().numpy()
    if isinstance(gt, torch.Tensor):
        gt = gt.cpu().numpy()
    
    # 이진 분류에 대한 혼동 행렬
    tp = np.sum((pred == 1) & (gt == 1))
    fp = np.sum((pred == 1) & (gt == 0))
    fn = np.sum((pred == 0) & (gt == 1))
    tn = np.sum((pred == 0) & (gt == 0))
    
    cm = np.array([[tn, fp], [fn, tp]])
    
    return cm

def calculate_metrics_from_cm(cm):
    """
    혼동 행렬에서 다양한 메트릭 계산
    
    Args:
        cm: 혼동 행렬 (2x2)
        
    Returns:
        metrics: 계산된 메트릭들
    """
    tn, fp = cm[0]
    fn, tp = cm[1]
    
    # 정확도 (Accuracy)
    accuracy = (tp + tn) / (tp + tn + fp + fn) if (tp + tn + fp + fn) > 0 else 0
    
    # 정밀도 (Precision)
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    
    # 재현율 (Recall) / 민감도 (Sensitivity)
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    
    # F1 점수
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    # 특이도 (Specificity)
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    
    # IoU (Intersection over Union) for class 1
    iou = tp / (tp + fp + fn) if (tp + fp + fn) > 0 else 0
    
    # 다이스 계수 (Dice Coefficient)
    dice = 2 * tp / (2 * tp + fp + fn) if (2 * tp + fp + fn) > 0 else 0
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'specificity': specificity,
        'iou': iou,
        'dice': dice,
    }

def process_all_fold_results(result_dir, modalities, n_folds, save_dir="results_analysis"):
    """
    모든 폴드 결과 처리 및 분석
    
    Args:
        result_dir: 결과 저장 디렉토리
        modalities: 모달리티 리스트
        n_folds: 폴드 수
        save_dir: 저장 디렉토리
    
    Returns:
        fold_results: 모든 폴드의 결과
    """
    fold_results = {}
    
    # 결과 저장 디렉토리 생성
    os.makedirs(save_dir, exist_ok=True)
    
    # 모든 모달리티 및 폴드 결과 수집
    for modality in modalities:
        fold_results[modality] = {}
        
        for fold in range(1, n_folds + 1):
            fold_dir = os.path.join(result_dir, f"{modality}_fold{fold}")
            result_file = os.path.join(fold_dir, f"fold{fold}_results_results.json")
            
            if os.path.exists(result_file):
                with open(result_file, 'r') as f:
                    # 첫 번째 항목 사용 (리스트 형태일 경우 대비)
                    fold_data = json.load(f)
                    if isinstance(fold_data, list) and len(fold_data) > 0:
                        fold_data = fold_data[0]
                
                fold_results[modality][fold] = fold_data
    
    # 모달리티 및 폴드별 요약 테이블 생성
    summary_data = []
    
    for modality in modalities:
        for fold in range(1, n_folds + 1):
            if fold in fold_results[modality]:
                data = fold_results[modality][fold]
                
                row = {
                    'modality': modality,
                    'fold': fold,
                    'val_dice': data.get('val_dice', 0),
                    'test_dice': data.get('test_dice', 0),
                    'best_epoch': data.get('best_epoch', 0),
                    'test_loss': data.get('test_loss', 0),
                }
                
                summary_data.append(row)
    
    # DataFrame 생성 및 저장
    summary_df = pd.DataFrame(summary_data)
    summary_path = os.path.join(save_dir, "fold_results_summary.csv")
    summary_df.to_csv(summary_path, index=False)
    
    # 결과 요약 시각화
    plt.figure(figsize=(12, 6))
    
    for modality in modalities:
        modality_data = summary_df[summary_df['modality'] == modality]
        if not modality_data.empty:
            plt.plot(modality_data['fold'], modality_data['test_dice'], 'o-', label=f"{modality}")
    
    plt.xlabel('Fold')
    plt.ylabel('Test Dice Score')
    plt.title('Test Dice Scores by Modality and Fold')
    plt.grid(True)
    plt.legend()
    plt.savefig(os.path.join(save_dir, "fold_test_dice_comparison.png"), dpi=300, bbox_inches='tight')
    plt.close()
    
    # 모달리티별 평균 성능 계산 및 시각화
    plt.figure(figsize=(10, 6))
    
    modality_avg = summary_df.groupby('modality')['test_dice'].mean().reset_index()
    modality_std = summary_df.groupby('modality')['test_dice'].std().reset_index()
    
    x = np.arange(len(modality_avg))
    width = 0.7
    
    plt.bar(x, modality_avg['test_dice'], width, yerr=modality_std['test_dice'],
            alpha=0.7, capsize=10, label='Avg Test Dice')
    
    plt.xlabel('Modality')
    plt.ylabel('Average Test Dice Score')
    plt.title('Average Test Dice Scores by Modality')
    plt.xticks(x, modality_avg['modality'])
    plt.grid(True, axis='y', alpha=0.3)
    plt.savefig(os.path.join(save_dir, "modality_avg_test_dice.png"), dpi=300, bbox_inches='tight')
    plt.close()
    
    return fold_results

def visualize_all_test_frames(base_dir, result_dir, test_patient_ids, modalities, n_folds, save_dir="visualized_results"):
    """
    모든 테스트 프레임 시각화
    
    Args:
        base_dir: 데이터셋 기본 경로
        result_dir: 결과 저장 디렉토리
        test_patient_ids: 테스트 환자 ID 목록
        modalities: 모달리티 리스트
        n_folds: 폴드 수
        save_dir: 저장 디렉토리
        
    Returns:
        results_by_modality_fold: 모달리티별, 폴드별 결과
    """
    # 기본 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    target_size = (224, 224)
    
    # 변환 설정
    test_transform = A.Compose([
        A.Resize(224, 224),
        ToTensorV2(),
    ])
    
    # 환자 프레임 수집
    print("환자 프레임 수집 중...")
    patient_frames = collect_patient_frames(
        base_dir, test_patient_ids, min_frame=20, max_frame=138
    )
    
    # 각 모달리티별 테스트 쌍 생성
    modality_test_pairs = {}
    for modality in modalities:
        print(f"{modality} 모달리티 테스트 쌍 생성 중...")
        test_pairs = create_image_mask_pairs(patient_frames, base_dir, modality=modality)
        modality_test_pairs[modality] = test_pairs
    
    # 저장 디렉토리 생성
    for modality in modalities:
        for fold in range(1, n_folds + 1):
            os.makedirs(os.path.join(save_dir, f"{modality}_fold{fold}"), exist_ok=True)
    
    # 각 모달리티 및 폴드에 대해 테스트 데이터에 대한 예측 및 시각화
    results_by_modality_fold = {}
    
    for modality in modalities:
        results_by_modality_fold[modality] = {}
        test_pairs = modality_test_pairs[modality]
        
        # 테스트 데이터셋 및 로더 생성
        test_dataset = MRIDataset(test_pairs, transform=test_transform, target_size=(224, 224))
        test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4, pin_memory=True)
        
        print(f"{modality} 모달리티 평가 중...")
        
        # 환자별로 결과를 구성하기 위한 구조 생성
        patient_frame_results = {}
        
        for fold in range(1, n_folds + 1):
            fold_dir = os.path.join(result_dir, f"{modality}_fold{fold}")
            model_path = os.path.join(fold_dir, "best_model.pth")
            
            if os.path.exists(model_path):
                print(f"  Fold {fold} 모델 로드 중...")
                model = load_model(model_path, device)
                
                print(f"  Fold {fold} 테스트 데이터 평가 중...")
                results = evaluate_model_on_test_data(model, test_loader, device)
                
                results_by_modality_fold[modality][fold] = results
                
                # 결과 시각화 및 저장
                print(f"  Fold {fold} 결과 시각화 중...")
                fold_save_dir = os.path.join(save_dir, f"{modality}_fold{fold}")
                
                # 환자별로 디렉토리 생성
                for result in results['individual_results']:
                    patient_id = result['patient_id']
                    frame_num = result['frame_num']
                    
                    # 환자별 디렉토리 생성
                    patient_dir = os.path.join(fold_save_dir, patient_id)
                    os.makedirs(patient_dir, exist_ok=True)
                    
                    # 원본 이미지와 마스크 로드
                    image_path = result['image_path']
                    mask_path = result['mask_path']
                    
                    orig_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                    if orig_image is None:
                        print(f"경고: 이미지를 로드할 수 없습니다: {image_path}")
                        continue
                    
                    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
                    if mask is None:
                        print(f"경고: 마스크를 로드할 수 없습니다: {mask_path}")
                        continue
                    
                    if mask.max() > 1:
                        mask = (mask > 0).astype(np.uint8)
                    
                    # 크기 조정
                    if orig_image.shape != (224, 224):
                        orig_image = cv2.resize(orig_image, (224, 224))
                    if mask.shape != (224, 224):
                        mask = cv2.resize(mask, (224, 224), interpolation=cv2.INTER_NEAREST)
                    
                    pred = result['pred']
                    
                    # 예측 크기 확인 및 조정
                    if pred.shape != (224, 224):
                        pred = cv2.resize(pred.astype(np.uint8), (224, 224), interpolation=cv2.INTER_NEAREST)
                    
                    # Dice 점수 재계산
                    dice_score = calculate_dice_score(pred, mask)
                    
                    # 결과 시각화 및 저장
                    save_path = os.path.join(patient_dir, f"frame_{frame_num}.png")
                    visualize_results(orig_image, mask, pred, dice_score, 
                                     title=f"Patient {patient_id}, Frame {frame_num}",
                                     save_path=save_path)
                    
                    # 환자별 결과 구조에 저장
                    if patient_id not in patient_frame_results:
                        patient_frame_results[patient_id] = {}
                    
                    if frame_num not in patient_frame_results[patient_id]:
                        patient_frame_results[patient_id][frame_num] = {}
                    
                    # 예측 크기 확인 및 조정 후 저장
                    resized_pred = pred.copy() if pred.shape == (224, 224) else cv2.resize(pred.astype(np.uint8), (224, 224), interpolation=cv2.INTER_NEAREST)
                    
                    patient_frame_results[patient_id][frame_num][fold] = {
                        'pred': resized_pred,
                        'dice_score': dice_score,
                    }
                
                print(f"  Fold {fold} 평균 Dice 점수: {results['avg_dice_score']:.4f}")
            else:
                print(f"  Fold {fold} 모델 파일이 존재하지 않습니다: {model_path}")
        
        # 앙상블 결과 저장을 위한 디렉토리 생성
        ensemble_dir = os.path.join(save_dir, f"{modality}_ensemble")
        os.makedirs(ensemble_dir, exist_ok=True)
        
        # 모달리티 내 폴드 앙상블 계산 및 시각화
        print(f"{modality} 모달리티 내 앙상블 계산 중...")
        
        ensemble_results = []
        
        for patient_id, frames in patient_frame_results.items():
            # 환자별 디렉토리 생성
            patient_dir = os.path.join(ensemble_dir, patient_id)
            os.makedirs(patient_dir, exist_ok=True)
            
            for frame_num, fold_results in frames.items():
                # 유효한 폴드 예측이 있는 경우에만 앙상블 수행
                valid_folds = list(fold_results.keys())
                
                if valid_folds:
                    # 첫 번째 폴드 결과에서 마스크 경로 가져오기
                    image_path = None
                    mask_path = None
                    
                    for result in results_by_modality_fold[modality][valid_folds[0]]['individual_results']:
                        if result['patient_id'] == patient_id and result['frame_num'] == frame_num:
                            image_path = result['image_path']
                            mask_path = result['mask_path']
                            break
                    
                    if image_path and mask_path:
                        # 원본 이미지와 마스크 로드
                        orig_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                        if orig_image is None:
                            print(f"경고: 이미지를 로드할 수 없습니다: {image_path}")
                            continue
                        
                        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
                        if mask is None:
                            print(f"경고: 마스크를 로드할 수 없습니다: {mask_path}")
                            continue
                        
                        if mask.max() > 1:
                            mask = (mask > 0).astype(np.uint8)
                        
                        # 크기 조정
                        if orig_image.shape != (224, 224):
                            orig_image = cv2.resize(orig_image, (224, 224))
                        if mask.shape != (224, 224):
                            mask = cv2.resize(mask, (224, 224), interpolation=cv2.INTER_NEAREST)
                        
                        # 앙상블 계산 (다수결 투표)
                        ensemble_pred = np.zeros((224, 224), dtype=np.uint8)
                        votes = np.zeros((224, 224), dtype=np.float32)
                        
                        for fold in valid_folds:
                            pred = fold_results[fold]['pred']
                            # 크기 확인 및 조정
                            if pred.shape != (224, 224):
                                pred = cv2.resize(pred.astype(np.uint8), (224, 224), interpolation=cv2.INTER_NEAREST)
                            votes += pred
                        
                        # 과반수 투표 (클래스 1의 표가 폴드 수의 절반을 초과하면 클래스 1로 예측)
                        ensemble_pred[votes > len(valid_folds) / 2] = 1
                        
                        # Dice 점수 계산
                        ensemble_dice = calculate_dice_score(ensemble_pred, mask)
                        
                        # 결과 시각화 및 저장
                        save_path = os.path.join(patient_dir, f"frame_{frame_num}_ensemble.png")
                        visualize_results(orig_image, mask, ensemble_pred, ensemble_dice, 
                                         title=f"Patient {patient_id}, Frame {frame_num} - Ensemble",
                                         save_path=save_path)
                        
                        # 앙상블 결과 저장
                        ensemble_results.append({
                            'patient_id': patient_id,
                            'frame_num': frame_num,
                            'image_path': image_path,
                            'mask_path': mask_path,
                            'dice_score': ensemble_dice,
                            'pred': ensemble_pred,
                        })
        
        if ensemble_results:
            # 앙상블 평균 Dice 점수 계산
            avg_ensemble_dice = np.mean([r['dice_score'] for r in ensemble_results])
            print(f"{modality} 모달리티 앙상블 평균 Dice 점수: {avg_ensemble_dice:.4f}")
            
            # 앙상블 결과 저장
            results_by_modality_fold[modality]['ensemble'] = {
                'individual_results': ensemble_results,
                'avg_dice_score': avg_ensemble_dice,
            }
        else:
            print(f"{modality} 모달리티에 대한 앙상블 결과가 없습니다.")
            results_by_modality_fold[modality]['ensemble'] = {
                'individual_results': [],
                'avg_dice_score': 0.0,
            }
    
    return results_by_modality_fold

def ensemble_across_modalities(results_by_modality_fold, modalities, save_dir="cross_modality_ensemble"):
    """
    모달리티 간 앙상블 수행
    
    Args:
        results_by_modality_fold: 모달리티별, 폴드별 결과
        modalities: 모달리티 리스트
        save_dir: 저장 디렉토리
    
    Returns:
        cross_modality_results: 모달리티 간 앙상블 결과
    """
    print("모달리티 간 앙상블 수행 중...")
    
    # 저장 디렉토리 생성
    os.makedirs(save_dir, exist_ok=True)
    
    # 모든 환자 및 프레임 수집
    all_patient_frames = {}
    
    for modality in modalities:
        if 'ensemble' not in results_by_modality_fold[modality]:
            print(f"경고: {modality} 모달리티에 앙상블 결과가 없습니다. 건너뜁니다.")
            continue
            
        ensemble_results = results_by_modality_fold[modality]['ensemble']['individual_results']
        
        for result in ensemble_results:
            patient_id = result['patient_id']
            frame_num = result['frame_num']
            
            if patient_id not in all_patient_frames:
                all_patient_frames[patient_id] = {}
            
            if frame_num not in all_patient_frames[patient_id]:
                all_patient_frames[patient_id][frame_num] = {}
            
            all_patient_frames[patient_id][frame_num][modality] = {
                'pred': result['pred'],
                'dice_score': result['dice_score'],
                'image_path': result['image_path'],
                'mask_path': result['mask_path'],
            }
    
    # 모달리티 조합 정의
    modality_combinations = []
    
    # 개별 모달리티 (이미 앙상블 결과로 계산됨)
    for m in modalities:
        if m in [mod for p in all_patient_frames.values() for f in p.values() for mod in f.keys()]:
            modality_combinations.append([m])
    
    # 2개씩 조합
    for i, m1 in enumerate(modalities):
        for m2 in modalities[i+1:]:
            if all(m in [mod for p in all_patient_frames.values() for f in p.values() for mod in f.keys()] for m in [m1, m2]):
                modality_combinations.append([m1, m2])
    
    # 3개씩 조합 (4개의 모달리티가 있는 경우)
    if len(modalities) >= 3:
        for i, m1 in enumerate(modalities):
            for j, m2 in enumerate(modalities[i+1:]):
                m2_idx = i + 1 + j  # 실제 m2의 인덱스
                for k, m3 in enumerate(modalities[m2_idx+1:]):
                    if all(m in [mod for p in all_patient_frames.values() for f in p.values() for mod in f.keys()] for m in [m1, m2, m3]):
                        modality_combinations.append([m1, m2, m3])
    
    # 모든 모달리티 조합
    if len(modalities) > 1 and all(m in [mod for p in all_patient_frames.values() for f in p.values() for mod in f.keys()] for m in modalities):
        modality_combinations.append(modalities)
    
    # 각 조합에 대한 앙상블 결과
    cross_modality_results = {}
    
    for combo in modality_combinations:
        combo_name = "_".join(combo)
        combo_dir = os.path.join(save_dir, combo_name)
        os.makedirs(combo_dir, exist_ok=True)
        
        print(f"앙상블 조합 계산 중: {combo_name}")
        
        ensemble_results = []
        
        for patient_id, frames in all_patient_frames.items():
            # 환자별 디렉토리 생성
            patient_dir = os.path.join(combo_dir, patient_id)
            os.makedirs(patient_dir, exist_ok=True)
            
            for frame_num, modality_results in frames.items():
                # 현재 조합에 포함된 모달리티만 사용
                valid_modalities = [m for m in combo if m in modality_results]
                
                if len(valid_modalities) == len(combo):
                    # 첫 번째 모달리티에서 이미지 및 마스크 경로 가져오기
                    image_path = modality_results[valid_modalities[0]]['image_path']
                    mask_path = modality_results[valid_modalities[0]]['mask_path']
                    
                    # 원본 이미지와 마스크 로드
                    orig_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                    if orig_image is None:
                        print(f"경고: 이미지를 로드할 수 없습니다: {image_path}")
                        continue
                        
                    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
                    if mask is None:
                        print(f"경고: 마스크를 로드할 수 없습니다: {mask_path}")
                        continue
                    
                    if mask.max() > 1:
                        mask = (mask > 0).astype(np.uint8)
                    
                    # 크기 조정
                    if orig_image.shape != (224, 224):
                        orig_image = cv2.resize(orig_image, (224, 224))
                    if mask.shape != (224, 224):
                        mask = cv2.resize(mask, (224, 224), interpolation=cv2.INTER_NEAREST)
                    
                    # 앙상블 계산 (다수결 투표)
                    ensemble_pred = np.zeros((224, 224), dtype=np.uint8)
                    votes = np.zeros((224, 224), dtype=np.float32)
                    
                    for modality in valid_modalities:
                        pred = modality_results[modality]['pred']
                        # 크기 확인 및 조정
                        if pred.shape != (224, 224):
                            pred = cv2.resize(pred.astype(np.uint8), (224, 224), interpolation=cv2.INTER_NEAREST)
                        votes += pred
                    
                    # 과반수 투표 (클래스 1의 표가 모달리티 수의 절반을 초과하면 클래스 1로 예측)
                    ensemble_pred[votes > len(valid_modalities) / 2] = 1
                    
                    # Dice 점수 계산
                    ensemble_dice = calculate_dice_score(ensemble_pred, mask)
                    
                    # 결과 시각화 및 저장
                    save_path = os.path.join(patient_dir, f"frame_{frame_num}_ensemble.png")
                    visualize_results(orig_image, mask, ensemble_pred, ensemble_dice, 
                                     title=f"Patient {patient_id}, Frame {frame_num} - {combo_name}",
                                     save_path=save_path)
                    
                    # 앙상블 결과 저장
                    ensemble_results.append({
                        'patient_id': patient_id,
                        'frame_num': frame_num,
                        'image_path': image_path,
                        'mask_path': mask_path,
                        'dice_score': ensemble_dice,
                        'pred': ensemble_pred,
                    })
        
        if ensemble_results:
            # 앙상블 평균 Dice 점수 계산
            avg_ensemble_dice = np.mean([r['dice_score'] for r in ensemble_results])
            print(f"{combo_name} 앙상블 평균 Dice 점수: {avg_ensemble_dice:.4f}")
            
            # 앙상블 결과 저장
            cross_modality_results[combo_name] = {
                'individual_results': ensemble_results,
                'avg_dice_score': avg_ensemble_dice,
                'modalities': combo,
            }
        else:
            print(f"{combo_name} 앙상블에 대한 결과가 없습니다.")
    
    if not cross_modality_results:
        print("모달리티 조합에 대한 앙상블 결과가 없습니다.")
        return {}
    
    # 모달리티 조합 간 성능 비교 시각화
    combo_names = list(cross_modality_results.keys())
    avg_dices = [cross_modality_results[name]['avg_dice_score'] for name in combo_names]
    
    # 성능 순으로 정렬
    sorted_indices = np.argsort(avg_dices)[::-1]  # 내림차순으로 정렬하기 위해 역순으로
    combo_names = [combo_names[i] for i in sorted_indices]
    avg_dices = [avg_dices[i] for i in sorted_indices]
    
    plt.figure(figsize=(14, 8))
    bars = plt.barh(combo_names, avg_dices, color='skyblue')
    
    # 각 막대에 값 표시
    for i, bar in enumerate(bars):
        plt.text(bar.get_width() + 0.005, bar.get_y() + bar.get_height()/2, 
                f"{avg_dices[i]:.4f}", va='center')
    
    plt.xlabel('Average Dice Score')
    plt.title('Cross-Modality Ensemble Performance Comparison')
    plt.grid(axis='x', alpha=0.3)
    plt.tight_layout()
    plt.savefig(os.path.join(save_dir, "cross_modality_ensemble_comparison.png"), dpi=300, bbox_inches='tight')
    plt.close()
    
    # 결과를 CSV로 저장
    combo_data = []
    for combo_name, results in cross_modality_results.items():
        combo_data.append({
            'combination': combo_name,
            'modalities': ", ".join(results['modalities']),
            'avg_dice_score': results['avg_dice_score'],
            'num_images': len(results['individual_results']),
        })
    
    combo_df = pd.DataFrame(combo_data)
    combo_df = combo_df.sort_values('avg_dice_score', ascending=False)
    combo_df.to_csv(os.path.join(save_dir, "cross_modality_ensemble_results.csv"), index=False)
    
    return cross_modality_results

def main():
    # 기본 설정
    base_dir = "./Dataset_both"
    result_dir = "./mri_segmentation_results_dice08_ce02"
    vis_save_dir = "./visualized_results"
    analysis_save_dir = "./results_analysis"
    cross_ensemble_dir = "./cross_modality_ensemble"
    
    # 테스트 환자 IDs
    test_patient_ids = ["35482165", "45209754", "70487644", "71424242", "38904284", "34003228"]
    
    # 모달리티 및 폴드 수
    modalities = ["t1", "t2", "junction", "extension"]
    n_folds = 5
    
    # 랜덤 시드 설정
    random_seed = 42
    random.seed(random_seed)
    np.random.seed(random_seed)
    torch.manual_seed(random_seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(random_seed)
    
    # GPU 확인
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    # 1. 모든 폴드 결과 처리 및 분석
    print("\n1. 폴드 결과 처리 및 분석 중...")
    fold_results = process_all_fold_results(
        result_dir=result_dir,
        modalities=modalities,
        n_folds=n_folds,
        save_dir=analysis_save_dir
    )
    
    # 2. 모든 테스트 프레임 시각화 및 앙상블 (모달리티 내)
    print("\n2. 테스트 프레임 시각화 및 앙상블 중...")
    modality_results = visualize_all_test_frames(
        base_dir=base_dir,
        result_dir=result_dir,
        test_patient_ids=test_patient_ids,
        modalities=modalities,
        n_folds=n_folds,
        save_dir=vis_save_dir
    )
    
    # 3. 모달리티 간 앙상블
    print("\n3. 모달리티 간 앙상블 중...")
    cross_modality_results = ensemble_across_modalities(
        results_by_modality_fold=modality_results,
        modalities=modalities,
        save_dir=cross_ensemble_dir
    )
    
    print("\n모든 결과 분석 및 시각화가 완료되었습니다!")
    print(f"- 폴드 결과 분석: {analysis_save_dir}")
    print(f"- 개별 모달리티 시각화: {vis_save_dir}")
    print(f"- 모달리티 간 앙상블: {cross_ensemble_dir}")

if __name__ == "__main__":
    main()

/home/temp_id/.local/lib/python3.10/site-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.6' (you have '2.0.4'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


Using device: cuda

1. 폴드 결과 처리 및 분석 중...

2. 테스트 프레임 시각화 및 앙상블 중...
환자 프레임 수집 중...
t1 모달리티 테스트 쌍 생성 중...
t2 모달리티 테스트 쌍 생성 중...
junction 모달리티 테스트 쌍 생성 중...
extension 모달리티 테스트 쌍 생성 중...
t1 모달리티 평가 중...
  Fold 1 모델 로드 중...
  Fold 1 테스트 데이터 평가 중...


Evaluating model: 100%|██████████| 7/7 [00:01<00:00,  6.38it/s]


  Fold 1 결과 시각화 중...
  Fold 1 평균 Dice 점수: 0.2363
  Fold 2 모델 로드 중...
  Fold 2 테스트 데이터 평가 중...


Evaluating model: 100%|██████████| 7/7 [00:00<00:00,  8.40it/s]


...

  Fold 3 결과 시각화 중...
  Fold 3 평균 Dice 점수: 0.2767
  Fold 4 모델 로드 중...
  Fold 4 테스트 데이터 평가 중...


Evaluating model: 100%|██████████| 7/7 [00:00<00:00,  7.07it/s]


  Fold 4 결과 시각화 중...
  Fold 4 평균 Dice 점수: 0.3368
  Fold 5 모델 로드 중...
  Fold 5 테스트 데이터 평가 중...


Evaluating model: 100%|██████████| 7/7 [00:01<00:00,  5.95it/s]


  Fold 5 결과 시각화 중...
